# FOR SORGENIA

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
import os
from numpy import load

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter

In [4]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils

In [5]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [6]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [7]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

Selecting GPU ID=0


In [8]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_final.csv'

In [9]:
raw_data: DataFrame = pd.read_csv(file_path)

In [10]:
raw_data.head()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2017-01-01 00:00:00,900.0,-0.783022,4.890773,1028.693079,983.524042,-1.734723e-15,1.278206,2.224647,...,0,UP_MPNTLCDMRN_1,0,1,6,1,UP_MPNTLCDMRN_1,0.0,6,0
1,UP_MPNTLCDMRN_1,2017-01-01 01:00:00,262.5,-0.565645,4.807044,1028.156679,983.034144,-1.734723e-15,1.211802,2.130597,...,0,UP_MPNTLCDMRN_1,1,1,6,1,UP_MPNTLCDMRN_1,1.0,6,1
2,UP_MPNTLCDMRN_1,2017-01-01 02:00:00,127.5,-0.331075,4.911959,1028.162779,983.062126,-1.734723e-15,1.017035,1.846413,...,0,UP_MPNTLCDMRN_1,2,1,6,1,UP_MPNTLCDMRN_1,2.0,6,2
3,UP_MPNTLCDMRN_1,2017-01-01 03:00:00,0.0,-0.668040,4.464566,1027.616920,982.551146,-1.734723e-15,0.715094,1.450024,...,0,UP_MPNTLCDMRN_1,3,1,6,1,UP_MPNTLCDMRN_1,3.0,6,3
4,UP_MPNTLCDMRN_1,2017-01-01 04:00:00,0.0,-0.617584,4.235597,1026.998009,981.962876,6.438134e-05,0.621238,1.329532,...,0,UP_MPNTLCDMRN_1,4,1,6,1,UP_MPNTLCDMRN_1,4.0,6,4


In [11]:
raw_data['time'] = raw_data['time'].astype('datetime64[s]')

In [12]:
raw_data.tail()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
245443,UP_PRCLCMINEO_1,2020-12-31 19:00:00,26688.0,3.4,5.7,1016.8,971.3,0.0,3.2,6.5,...,1460,UP_PRCLCMINEO_1,19,31,3,12,UP_PRCLCMINEO_1,35059.0,3,19
245444,UP_PRCLCMINEO_1,2020-12-31 20:00:00,20178.0,3.1,5.1,1017.0,971.5,0.0,2.8,5.8,...,1460,UP_PRCLCMINEO_1,20,31,3,12,UP_PRCLCMINEO_1,35060.0,3,20
245445,UP_PRCLCMINEO_1,2020-12-31 21:00:00,16074.0,2.7,4.7,1017.3,971.7,0.0,2.5,5.2,...,1460,UP_PRCLCMINEO_1,21,31,3,12,UP_PRCLCMINEO_1,35061.0,3,21
245446,UP_PRCLCMINEO_1,2020-12-31 22:00:00,10712.4,2.3,4.3,1017.5,971.9,0.0,2.4,4.9,...,1460,UP_PRCLCMINEO_1,22,31,3,12,UP_PRCLCMINEO_1,35062.0,3,22
245447,UP_PRCLCMINEO_1,2020-12-31 23:00:00,7211.4,1.8,4.0,1017.4,971.9,0.0,2.4,4.7,...,1460,UP_PRCLCMINEO_1,23,31,3,12,UP_PRCLCMINEO_1,35063.0,3,23


WE'RE MISSING 2021 DATA